## Score challenge submission

This notebook produces scores for a custom challenge set.  It scores each tasks and provides a summary score (average).

This is designed to help compare the quality of a solution against a challenge set under our control and demonstrate that the aicrowd score is consistent.

In [ ]:
import sys
import json
import re
import collections
import os
import datetime
import pandas as pd
import numpy as np
import math

In [ ]:
challenge="ex2-from-21k-train-with-pids"
submitdir="/home/jpr/projects/mpd-challenge-aicrowd/"
submissionfile="method-01-mympd-2nd-21k-2021-11-14.csv.gz"
submissions = [{"tag": "vl6", "file": submissionfile}]
submissionfile="method-02-mympd-2nd-21k-2021-11-07.csv.gz"
submissions.append({"tag": "hw", "file": submissionfile})

### Load the no holdouts challenge set

In [ ]:
with open('data/{}/challenge_set_noholdout.json'.format(challenge),'r') as f:
    data = json.loads(f.read())

In [ ]:
noholdout = pd.json_normalize(data,"playlists")

In [ ]:
noholdout.head()

### Load challenge submission`

In [ ]:
rectracks=pd.read_csv('{}/{}'.format(submitdir, submissionfile), header=None, skiprows=1, index_col=0, skipinitialspace=True)


Make sure the pids match those of the challenge set.  In the case of mympd the pid range starts at 200000 and goes up in sequence of noholdout data set.

In [ ]:
rectracks.head()

In [ ]:
rectracks = rectracks.sort_values(by=0, axis='index')

In [ ]:
rectracks.head()

## Score Challenge Submission

In [ ]:
startpid=2000000

R-precision is the fraction of correctly recommended tracks in the ground truth playlist as described on [the challenge site](https://www.aicrowd.com/challenges/spotify-million-playlist-dataset-challenge#evaluation)
    

Use the [r-precesion calculation from the hello_world metrics](https://github.com/jprorama/spotify_recSys_challenge_2018/blob/f33d82715190a20fdbc998c9ff709bcabd62a55e/utils/metrics.py#L26)

In [ ]:
def get_r_precision(answer, cand):
    set_answer = set(answer)
    r = len(set_answer&set(cand[:len(set_answer)])) / len(set_answer)
    return r

def get_ndcg(answer, cand):
    cand_len = len(cand) 
    idcg=0
    dcg=0
    
    #print("cand len {}".format(cand_len))
    #print("ans len {}".format(len(answer)))
    #print("cand: {}".format(cand))
    
    for i in range(cand_len):
        #print("i {}".format(i))
        #print("cand {}".format(cand[i]))
        if cand[i] in answer: 
            dcg += (1/math.log(i+1+1,2))

    for i in range(len(set(answer))):
        idcg += (1/math.log(i+1+1,2))
    
    return dcg/idcg

In [ ]:
realtracks=pd.DataFrame()
rprec_sum = 0.0
ndcg_sum = 0.0
rprec_match = 0
ndcg_match = 0

for i in range(len(noholdout)):
    pid = startpid + i
    #print("pid={}".format(pid))
    gttracks = [track["track_uri"] for track in noholdout[noholdout["challenge_pid"]==pid].tracks.to_list()[0]]
    candtracks = rectracks.loc[pid].to_list()
    rprec = get_r_precision(gttracks, candtracks)
    rprec_sum = rprec_sum + rprec
    if rprec > 0:
        rprec_match += 1
    ndcg = get_ndcg(gttracks, candtracks)
    ndcg_sum = ndcg_sum + ndcg
    if ndcg > 0:
        ndcg_match += 1
    
print("rprec = {}".format(rprec_sum/len(noholdout)))
print("ndcg = {}".format(ndcg_sum/len(noholdout)))
print("rprec_match = {}".format(rprec_match))
print("ndcg_match = {}".format(ndcg_match))

In [ ]:
def score_set(answer, candidates):
    rprec_sum = 0.0
    ndcg_sum = 0.0
    rprec_match = 0
    ndcg_match = 0

    scores = dict()

    for pid in answer.challenge_pid:
        #pid = startpid + i
        #print("pid={}".format(pid))
        gttracks = [track["track_uri"] for track in answer[answer["challenge_pid"]==pid].tracks.to_list()[0]]
        candtracks = candidates.loc[pid].to_list()
        rprec = get_r_precision(gttracks, candtracks)
        rprec_sum = rprec_sum + rprec
        if rprec > 0:
            rprec_match += 1
        ndcg = get_ndcg(gttracks, candtracks)
        ndcg_sum = ndcg_sum + ndcg
        if ndcg > 0:
            ndcg_match += 1
        
    scores["rprec"] = rprec_sum/len(answer)
    scores["rprec_match"] = rprec_match
    scores["ndcg"] = ndcg_sum/len(answer)
    scores["ndcg_match"] = ndcg_match
        
    return scores

In [ ]:
rprec_sum = 0.0
ndcg_sum = 0.0
rprec_match = 0
ndcg_match = 0

scores = dict()

for task in noholdout.task_name.drop_duplicates():

    scores[task] = score_set(noholdout[noholdout["task_name"]==task], rectracks)
    
    rprec_sum += scores[task]["rprec"]
    rprec_match += scores[task]["rprec_match"]
    ndcg_sum += scores[task]["ndcg"]
    ndcg_match += scores[task]["ndcg_match"]
    
    
    print("task: {}".format(task))
    print("rprec = {}".format(scores[task]["rprec"]))
    print("ndcg = {}".format(scores[task]["ndcg"]))
    print("rprec_match = {}".format(scores[task]["rprec_match"]))
    print("ndcg_match = {}".format(scores[task]["ndcg_match"]))
    print("\n")

scores["total"] = dict()
scores["total"]["rprec"] = rprec_sum/10
scores["total"]["rprec_match"] = rprec_match
scores["total"]["ndcg"] = ndcg_sum/10
scores["total"]["ndcg_match"] = ndcg_match

print("task: {}".format("total"))
print("rprec = {}".format(scores["total"]["rprec"]))
print("ndcg = {}".format(scores["total"]["ndcg"]))
print("rprec_match = {}".format(scores["total"]["rprec_match"]))
print("ndcg_match = {}".format(scores["total"]["ndcg_match"]))
print("\n")

In [ ]:
def score_setdf(answer, candidate):

    rprec_sum = 0.0
    ndcg_sum = 0.0
    rprec_match = 0
    ndcg_match = 0

    results = answer.copy()

    #results["candidates"] = 
    results["rprec"] = 0
    results["ndcg"] = 0

    for pid in results.challenge_pid:
        #pid = startpid + i
        #print("pid={}".format(pid))
        gttracks = [track["track_uri"] for track in results[results["challenge_pid"]==pid].tracks.to_list()[0]]
        candtracks = rectracks.loc[pid].to_list()
        rprec = get_r_precision(gttracks, candtracks)
        rprec_sum = rprec_sum + rprec
        if rprec > 0:
            rprec_match += 1
        ndcg = get_ndcg(gttracks, candtracks)
        ndcg_sum = ndcg_sum + ndcg
        if ndcg > 0:
            ndcg_match += 1

        #results.at[results["challenge_pid"]==pid, "candidates"] = candtracks
        results.at[results["challenge_pid"]==pid, "rprec"] = rprec
        results.at[results["challenge_pid"]==pid, "ndcg"] = ndcg
        
    return results


In [ ]:
results = pd.DataFrame()
tmpdf = pd.DataFrame()

for submission in submissions:
    print("submission: {}".format(submission))
    print("file: {}".format(submission["file"]))
    rectracks=pd.read_csv('{}/{}'.format(submitdir, submission["file"]), header=None, skiprows=1, index_col=0, skipinitialspace=True)
    tmpdf = score_setdf(noholdout, rectracks)
    print("tmpdf len: {}".format(len(tmpdf)))
    tmpdf["tag"]=submission["tag"]
    results=results.append(tmpdf)

In [ ]:
results.groupby(["tag", "task_name"]).rprec.mean()

In [ ]:
nodup = results[results["tag"]=="vl6"].drop_duplicates(subset="pid")

In [ ]:
nodup = nodup.append(results[results["tag"]=="hw"].drop_duplicates(subset="pid"))

In [ ]:
nodup.groupby(["tag", "task_name"]).rprec.mean()

In [ ]:
means= nodup.groupby(["tag", "task_name"]).rprec.mean()

In [ ]:

ax=means.hw.plot.bar(legend=True, color="red", alpha=.5)
means.vl6.plot.bar(ax=ax,legend=True, alpha=0.35)


## Explore Violin plot

start with the routine used during the recsys18 analysis

In [ ]:
import seaborn as sns

In [ ]:
def plot_violin(df, title="Violin Plot"):
    #sns.set_style("white") 
    sns.set(rc={'figure.figsize':(8,6)})
    g = sns.violinplot(data=df, cut=0, orient='v', scale='width')
    #g = sns.violinplot(x=df.iloc[,0], y=df.iloc[0,:], cut=0, scale='width')
    g.set_title(title)
    #g.set_xlabel("Subtask")
    g.set_ylabel("Score")
    g.set_xticklabels(g.get_xticklabels(), rotation=90)

    return g

In [ ]:
nodup.groupby(["tag", "task_name"]).rprec.describe().T

Don't need the describe data because that will come from the violin plot.  Just need to use all the raw data points.

In [ ]:
vl6desc = nodup[nodup["tag"]=="vl6"][["task_name", "rprec"]] #.groupby(["task_name"]).mean()

In [ ]:
hwdesc = nodup[nodup["tag"]=="hw"][["task_name", "rprec"]]

In [ ]:
g = sns.violinplot(data=g, cut=0, orient='v', scale='width')

In [ ]:
plot_violin(vl6desc[vl6desc["task_name"]=="rand_100_title"], "rand_100_title")

In [ ]:
plot_violin(hwdesc[hwdesc["task_name"]=="rand_100_title"], "rand_100_title")

## Explore side-by-side plots

What to take all the tasks and see the teams side by side

Create violin plots for each task. https://stackoverflow.com/a/47487445/8928529

Basically loop through the tasks and plot on each subplot axis.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, axes = plt.subplots(10, 2, figsize=(20, 32), sharey='row')
axes_cols = (axes.flatten()[::2], axes.flatten()[1::2])

i=0
for task in vl6desc.task_name.drop_duplicates():
    
    ax=axes_cols[0][i]
    sns.violinplot(data=vl6desc[vl6desc["task_name"]==task], cut=0, orient='v', scale='width', ax=ax)
    ax.set_title('task = {}'.format(task), y=0.95)
    ax=axes_cols[1][i]
    sns.violinplot(data=hwdesc[hwdesc["task_name"]==task], cut=0, orient='v', scale='width', ax=ax)
    ax.set_title('task = {}'.format(task), y=0.95)
    
    
    i += 1

## Explore split plot

This lets me see the data paired directly and allows easier visual comparison of differences.

The test rand_100_title task shows clear differences.

In [ ]:
data = nodup[["tag","task_name", "rprec"]]

In [ ]:
sns.violinplot(data=data[data["task_name"]=="rand_100_title"], cut=0, orient='v', scale='width',
               x="task_name", y="rprec",
               hue="tag",
               split=True, inner="quart")

#sns.despine(left=True)

In [ ]:
rprec_data = nodup[["tag","task_name", "rprec"]]
ndcg_data  = nodup[["tag","task_name", "ndcg"]]

In [ ]:
fig, axes = plt.subplots(10, 2, figsize=(20, 32), sharey='row')
axes_cols = (axes.flatten()[::2], axes.flatten()[1::2])

i=0
for task in vl6desc.task_name.drop_duplicates():
    
    ax=axes_cols[0][i]
    sns.violinplot(data=rprec_data[rprec_data["task_name"]==task], cut=0, orient='v', scale='width',
               x="task_name", y="rprec",
               hue="tag",
               split=True, inner="quart", ax=ax)
    ax.set_title('task = {}'.format(task), y=0.95)

    ax=axes_cols[1][i]
    sns.violinplot(data=ndcg_data[ndcg_data["task_name"]==task], cut=0, orient='v', scale='width',
               x="task_name", y="ndcg",
               hue="tag",
               split=True, inner="quart", ax=ax)
    ax.set_title('task = {}'.format(task), y=0.95)
    
    
    i += 1

With the rprec and ndcg data plotted side by side it is clear that the rand_100_title was much more effectively solved by vl6.

The second most effective was first_1_title where vl6 had higher mean.

However the the title_only solution of hw is clearly better.

All the rest of the tasks had nearly identical means and distributions with the hw solution having slightly higher ndcg in those tasks.
